# Autogen உடன் Agentic RAG

இந்த நோட்ட்புக் Retrieval-Augmented Generation (RAG) ஐ Autogen முகவர்களுடன் மேம்பட்ட மதிப்பீட்டு திறன்களுடன் செயல்படுத்துவதைக் காட்டுகிறது.


SQLite பதிப்பு சரி செய்யல்  
நீங்கள் பின்வரும் பிழையை சந்தித்தால்:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```
  
உங்கள் நோட்புக் தொடக்கத்தில் இந்த குறியீட்டு தொகுதியை Uncomment செய்யவும்:  


In [2]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
!pip install chromadb

In [3]:
import os
import time
import asyncio
from typing import List, Dict
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient

import chromadb

load_dotenv()

True

## கிளையன்டை உருவாக்குதல்

முதலில், Azure AI Chat Completion Client ஐ தொடங்குகிறோம். இந்த கிளையன்ட், பயனர் கேள்விகளுக்கு பதில்களை உருவாக்க Azure OpenAI சேவையுடன் தொடர்பு கொள்ள பயன்படுத்தப்படும்.


In [4]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

## வெக்டர் தரவுத்தொகுப்பு தொடக்கம்

நாங்கள் ChromaDB-ஐ நிலையான சேமிப்புடன் தொடங்கி மேம்படுத்தப்பட்ட மாதிரித் ஆவணங்களைச் சேர்க்கிறோம். ChromaDB ஆவணங்களைச் சேமிக்கவும், சரியான பதில்களை உருவாக்க உதவும் சூழலை மீட்டெடுக்கவும் பயன்படுத்தப்படும்.


In [5]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# Enhanced sample documents
documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours."
]

# Add documents with metadata
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

## Context Provider செயல்பாடு

`ContextProvider` வகுப்பு பல்வேறு மூலங்களிலிருந்து சூழலை பெறுவதையும் ஒருங்கிணைப்பதையும் கையாளுகிறது:

1. **வெக்டர் தரவுத்தொகுப்பு பெறுதல்**: ChromaDB-ஐ பயன்படுத்தி பயண ஆவணங்களில் அர்த்தமுள்ள தேடலைச் செய்கிறது
2. **வானிலை தகவல்**: முக்கிய நகரங்களுக்கு ஒரு ச模拟 வானிலை தரவுத்தொகுப்பை பராமரிக்கிறது
3. **ஒற்றுமையான சூழல்**: ஆவண மற்றும் வானிலை தரவுகளை ஒருங்கிணைத்து முழுமையான சூழலை உருவாக்குகிறது

முக்கியமான முறைமைகள்:
- `get_retrieval_context()`: கேள்வியின் அடிப்படையில் தொடர்புடைய ஆவணங்களைப் பெறுகிறது
- `get_weather_data()`: குறிப்பிட்ட இடங்களுக்கான வானிலை தகவலை வழங்குகிறது
- `get_unified_context()`: ஆவண மற்றும் வானிலை சூழல்களை ஒருங்கிணைத்து மேம்பட்ட பதில்களை உருவாக்குகிறது


In [6]:
class ContextProvider:
    def __init__(self, collection):
        self.collection = collection
        # Simulated weather database
        self.weather_database = {
            "new york": {"temperature": 72, "condition": "Partly Cloudy", "humidity": 65, "wind": "10 mph"},
            "london": {"temperature": 60, "condition": "Rainy", "humidity": 80, "wind": "15 mph"},
            "tokyo": {"temperature": 75, "condition": "Sunny", "humidity": 50, "wind": "5 mph"},
            "sydney": {"temperature": 80, "condition": "Clear", "humidity": 45, "wind": "12 mph"},
            "paris": {"temperature": 68, "condition": "Cloudy", "humidity": 70, "wind": "8 mph"},
        }
    
    def get_retrieval_context(self, query: str) -> str:
        """Retrieves relevant documents from vector database based on query."""
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_strings = []
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_strings.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_strings) if context_strings else "No relevant documents found"
    
    def get_weather_data(self, location: str) -> str:
        """Simulates retrieving weather data for a given location."""
        if not location:
            return ""
            
        location_key = location.lower()
        if location_key in self.weather_database:
            data = self.weather_database[location_key]
            return f"Weather for {location.title()}:\n" \
                   f"Temperature: {data['temperature']}°F\n" \
                   f"Condition: {data['condition']}\n" \
                   f"Humidity: {data['humidity']}%\n" \
                   f"Wind: {data['wind']}"
        else:
            return f"No weather data available for {location}."
    
    def get_unified_context(self, query: str, location: str = None) -> str:
        """Returns a unified context combining both document retrieval and weather data."""
        retrieval_context = self.get_retrieval_context(query)
        
        weather_context = ""
        if location:
            weather_context = self.get_weather_data(location)
            weather_intro = f"\nWeather Information for {location}:\n"
        else:
            weather_intro = ""
        
        return f"Retrieved Context:\n{retrieval_context}\n\n{weather_intro}{weather_context}"

## முகவர் அமைப்பு

நாங்கள் தேடல் மற்றும் உதவியாளர் முகவர்களை அமைக்கிறோம். தேடல் முகவர் அர்த்தமுள்ள தேடலைப் பயன்படுத்தி தொடர்புடைய தகவல்களை கண்டறிவதில் நிபுணத்துவம் பெற்றவர், அதே நேரத்தில் உதவியாளர் கண்டறியப்பட்ட தகவலின் அடிப்படையில் விரிவான பதில்களை உருவாக்குகிறார்.


In [7]:
# Create agents with enhanced capabilities
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message=(
        "You are a helpful AI assistant that provides answers using ONLY the provided context. "
        "Do NOT include any external information. Base your answer entirely on the context given below."
    ),
)

## RAG மூலம் கேள்வி செயலாக்கம்

நாங்கள் `ask_rag` செயல்பாட்டை வரையறுக்கிறோம், இது கேள்வியை உதவியாளருக்கு அனுப்பி, பதிலை செயலாக்கி, அதை மதிப்பீடு செய்யும். இந்த செயல்பாடு உதவியாளருடன் தொடர்பு கொள்ளும் மற்றும் பதிலின் தரத்தை அளவிடEvaluator ஐப் பயன்படுத்துகிறது.


In [8]:
async def ask_rag_agent(query: str, context_provider: ContextProvider, location: str = None):
    """
    Sends a query to the assistant agent with context from the provider.
    
    Args:
        query: The user's question
        context_provider: The context provider instance
        location: Optional location for weather queries
    """
    try:
        # Get unified context
        context = context_provider.get_unified_context(query, location)
        
        # Augment the query with context
        augmented_query = (
            f"{context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide a helpful answer."
        )

        # Send the augmented query to the assistant
        start_time = time.time()
        response = await assistant.on_messages(
            [TextMessage(content=augmented_query, source="user")],
            cancellation_token=CancellationToken(),
        )
        processing_time = time.time() - start_time
        
        return {
            'query': query,
            'response': response.chat_message.content,
            'processing_time': processing_time,
            'location': location
        }
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

# உதாரண பயன்பாடு

நாம் மதிப்பீட்டாளரை தொடங்குகிறோம் மற்றும் செயல்படுத்தி மதிப்பீடு செய்ய விரும்பும் கேள்விகளை வரையறுக்கிறோம்.


In [ ]:
async def main():
    # Initialize context provider
    context_provider = ContextProvider(collection)
    
    # Example queries
    queries = [
        {"query": "What does Contoso's travel insurance cover?"},
        {"query": "What's the weather like in London?", "location": "london"},
        {"query": "What luxury destinations does Contoso offer and what's the weather in Paris?", "location": "paris"},
    ]
    
    print("=== Autogen RAG Demo ===")
    for query_data in queries:
        query = query_data["query"]
        location = query_data.get("location")
        
        print(f"\n\nQuery: {query}")
        if location:
            print(f"Location: {location}")
        
        # Show the context being used
        context = context_provider.get_unified_context(query, location)
        print("\n--- Context Used ---")
        print(context)
        print("-------------------")
        
        # Get response from the agent
        result = await ask_rag_agent(query, context_provider, location)
        if result:
            print(f"\nResponse: {result['response']}")
        print("\n" + "="*50)

## ஸ்கிரிப்டை இயக்கவும்

ஸ்கிரிப்டு ஒரு இடையறை சூழலில் அல்லது ஒரு நிலையான ஸ்கிரிப்டில் இயங்குகிறதா என்பதை நாம் சரிபார்க்கிறோம், மற்றும் அதன்படி முக்கிய செயல்பாட்டை இயக்குகிறோம்.


In [12]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        await main()
    else:
        asyncio.run(main())

=== Autogen RAG Demo ===


Query: What does Contoso's travel insurance cover?

--- Context Used ---
Retrieved Context:
Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


-------------------

Response: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.



Query: What's the weather like in London?
Location: london

--- Context Used ---
Retrieved Context:
Document: Popular destinations include the Maldives, Swiss Alps, and African safaris.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


Weather Information for london:
Weather for London:
T


---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியக்க மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளுங்கள். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
